# INM433 - Individual coursework: Data pre-processing

In [95]:
import geopandas as gp
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
from ipywidgets import interact
import contextily as ctx
from sklearn.linear_model import LinearRegression

In [96]:
path = "C:\\Users\\Asus\\Documents\\City University\\INM433 Visual Analytics\\IndivualCoursework\\data\\coursework\\"

## Boro_precinct_arrest [1] 

In [97]:
boro_precinct_arrest = pd.read_csv( path + 'NYPD_Arrest_Data__Year_to_Date_.csv', sep = "," )

In [98]:
boro_precinct_arrest["ARREST_BORO"] = boro_precinct_arrest["ARREST_BORO"].replace( {"B":"Bronx",
                                                "K":"Brooklyn",
                                                "S":"Staten Island",
                                                "Q":"Queens",
                                                "M":"Manhattan"} )
boro_precinct_arrest.rename( columns = {"ARREST_PRECINCT":"precinct", "ARREST_BORO":"boro_name"}, inplace = True )

boro_precinct_arrest["ARREST_DATE"] = pd.to_datetime( boro_precinct_arrest["ARREST_DATE"], dayfirst = False )
boro_precinct_arrest["ARREST_DATE"] = pd.to_datetime(  boro_precinct_arrest["ARREST_DATE"] )
boro_precinct_arrest["MONTH"] = boro_precinct_arrest["ARREST_DATE"].apply( lambda x: x.month )
boro_precinct_arrest["SEASON"] = boro_precinct_arrest["MONTH"].apply( lambda x: ( x%12 + 3 )//3 )
boro_precinct_arrest["WEEKDAY"] = boro_precinct_arrest["ARREST_DATE"].apply( lambda x: x.weekday( ) )
boro_precinct_arrest["DAY"] = boro_precinct_arrest["ARREST_DATE"].apply( lambda x: x.day )

boro_precinct_arrest["WEEKDAY"].replace({0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}, inplace = True )
boro_precinct_arrest["SEASON"].replace({1:"Winter", 2:"Spring", 3:"Summer", 4:"Autumn"}, inplace = True )

In [99]:
boro_precinct_arrest.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,boro_name,precinct,...,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,MONTH,SEASON,WEEKDAY,DAY
0,203078287,2019-09-30,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,Manhattan,9,...,M,BLACK HISPANIC,990563,203120,40.724200,-73.977226,9,Autumn,Monday,30
1,203072424,2019-09-30,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,Queens,113,...,M,BLACK,1040611,190715,40.689974,-73.796769,9,Autumn,Monday,30
2,203061215,2019-09-30,905.0,"INTOXICATED DRIVING,ALCOHOL",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920U3,M,Staten Island,122,...,M,WHITE,962989,160112,40.606129,-74.076570,9,Autumn,Monday,30
3,203061218,2019-09-30,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601502,F,Staten Island,120,...,M,WHITE HISPANIC,962822,174282,40.645023,-74.077217,9,Autumn,Monday,30
4,203063729,2019-09-30,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,Queens,106,...,M,ASIAN / PACIFIC ISLANDER,1035521,188624,40.684266,-73.815138,9,Autumn,Monday,30


## Points of Interests table [2]

In [100]:
boro_PI = pd.read_csv(path+"Point_Of_Interest.csv", sep =",")

In [101]:
boro_PI["BOROUGH"] = boro_PI["BOROUGH"].replace( {2:"Bronx", 3:"Brooklyn",
                                            5:"Staten Island",
                                            4:"Queens",
                                            1:"Manhattan"} )

In [102]:
boro_PI[["Longitude","Latitude"]] = boro_PI["the_geom"].str.split(" ", expand = True).apply(lambda x: x.str.replace("(","").str.replace(")",""))[[1,2]]

In [103]:
boro_PI["the_geom"]= boro_PI.apply(lambda x : Point(float(x.loc["Longitude"]), float(x.loc["Latitude"]),), axis =1)

In [104]:
boro_PI.FACI_DOM = boro_PI.FACI_DOM.replace({1:"Residential",2:"Education",3:"Cultural",4:"Recreational",5:"Social",6:"Transportation",7:"Commercial",8:"Government",9:"Religious",10:"Health",11:"Safety",12:"Water",13:"Miscellaneous"})

## Schools table [3]

In [105]:
schools  = gp.read_file(path + "Schools\\Public_Schools_Points_2011-2012A.shp")
schools.rename(columns ={"City": "boro_name"}, inplace = True)

In [106]:
schools.boro_name = schools.boro_name.replace({"BRONX":"Bronx", "BROOKLYN":"Brooklyn", "MANHATTAN":"Manhattan", "QUEENS":"Queens", "STATEN ISLAND":"Staten Island"} )

In [107]:
schools = schools.to_crs( epsg = 4326 )

In [108]:
schools["geometry"] = schools["geometry"].apply(lambda x: str(x))

In [109]:
schools[["Longitude","Latitude"]] = schools["geometry"].str.split(" ", expand = True).apply(lambda x: x.str.replace("(","").str.replace(")",""))[[1,2]]

In [110]:
schools["geometry"]= schools.apply(lambda x : Point(float(x.loc["Longitude"]), float(x.loc["Latitude"])), axis =1)

## Districts shapefile [4]

In [111]:
districts = gp.read_file(path + "Community districts\\Community_districts.shp")
districts["Borough"]=districts.boro_cd.apply(lambda x : int(str(x)[0])).replace( {2:"Bronx", 3:"Brooklyn",
                                            5:"Staten Island",
                                            4:"Queens",
                                            1:"Manhattan"} )
districts["boro_cd"]=districts.boro_cd.apply(lambda x : int(str(x)[1:3]))

## Districts table

In [112]:
districts_table = districts[["Borough","boro_cd"]].copy()

### Population information [5]

In [113]:
population = pd.read_csv(path+ "New_York_City_Population_By_Community_Districts.csv", sep = ";")
population.iloc[:,3:]= population.iloc[:,3:].transform(lambda x : x.str.replace(",","") )
population.rename( columns = {"CD Number":"boro_cd"}, inplace = True)

In [114]:
population.head()

,Borough,boro_cd,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762
3,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441
4,Bronx,5,"University Hts., Fordham, Mt. Hope",121807,107995,118435,128313,128200


In [115]:
districts_table = districts_table[districts_table.boro_cd.isin(population.boro_cd.unique())]
districts = districts[districts.boro_cd.isin(population.boro_cd.unique())]

- What is the population in 2019?

In [116]:
population["Population_2019"] = population.apply(lambda x: LinearRegression().fit( np.array([1990,2000,2010]).reshape(-1, 1),x.iloc[5:]).predict(np.array([2019]).reshape(-1, 1))[0], axis = 1)

In [117]:
population.set_index(["Borough","boro_cd"], inplace = True)
districts_table.set_index(["Borough","boro_cd"], inplace = True)
districts.set_index(["Borough","boro_cd"], inplace = True)

In [118]:
districts_table = pd.concat([districts_table, population[["Population_2019", "CD Name"]]], ignore_index = False, axis = 1)

### Arrests information

In [119]:
boro_precinct_arrest["Points"]= boro_precinct_arrest.apply(lambda x : Point(x.loc["Longitude"],x.loc["Latitude"],), axis =1)

In [120]:
pnts = gp.GeoDataFrame(geometry=boro_precinct_arrest["Points"], index=boro_precinct_arrest.index)
pnts = pnts.assign(**{str(key): pnts.within(geom) for key, geom in districts.geometry.items()})

Drop points outside of the city

In [121]:
pnts.sum(axis=1)[pnts.sum(axis=1)==0].index

Int64Index([   140,    163,    216,    528,    768,    791,   1429,   1454,
              1594,   1799,
            ...
            165586, 165589, 165683, 165692, 165696, 166947, 166965, 167349,
            167364, 167381],
           dtype='int64', length=679)

In [122]:
boro_precinct_arrest.drop(index = pnts.sum(axis=1)[pnts.sum(axis=1)==0].index, inplace = True)
pnts.drop(index = pnts.sum(axis=1)[pnts.sum(axis=1)==0].index, inplace = True)

In [123]:
boro_precinct_arrest["boro_cd"] = pnts.apply(lambda x: eval(x[x==True].index[0])[1], axis =1)

In [124]:
districts_table["Arrest"] = boro_precinct_arrest.groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)

In [125]:
for sex in boro_precinct_arrest.PERP_SEX.unique():
    select = boro_precinct_arrest.PERP_SEX == sex
    districts_table["Arr_S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)

In [126]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
    select = boro_precinct_arrest.AGE_GROUP == age_group
    districts_table["Arr_A"+age_group] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
    districts_table["Arr_A"+age_group] = districts_table["Arr_A"+age_group].fillna(0)

In [127]:
for crime in ['F', 'M', 'V', 'I']:
    select = boro_precinct_arrest.LAW_CAT_CD == crime
    districts_table["Arr_C"+crime] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
    districts_table["Arr_C"+crime] = districts_table["Arr_C"+crime].fillna(0)

In [128]:
for race in boro_precinct_arrest.PERP_RACE.unique():
    select = boro_precinct_arrest.PERP_RACE == race
    districts_table["Arr_R"+race] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
    districts_table["Arr_R"+race] = districts_table["Arr_R"+race].fillna(0)

In [129]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
    for sex in boro_precinct_arrest.PERP_SEX.unique():
        for crime in ['F', 'M', 'V', 'I']:
            for race in boro_precinct_arrest.PERP_RACE.unique():
                select = (boro_precinct_arrest.PERP_SEX == sex)&(boro_precinct_arrest.AGE_GROUP == age_group)&(boro_precinct_arrest.LAW_CAT_CD == crime)&(boro_precinct_arrest.PERP_RACE == race)
                if boro_precinct_arrest[select].shape[0]>0:
                    districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race+"//S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                    districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race+"//S"+sex] = districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race+"//S"+sex].fillna(0)
                else:
                    districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race+"//S"+sex] = 0

In [130]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
    for sex in boro_precinct_arrest.PERP_SEX.unique():
            for race in boro_precinct_arrest.PERP_RACE.unique():
                select = (boro_precinct_arrest.PERP_SEX == sex)&(boro_precinct_arrest.AGE_GROUP == age_group)&(boro_precinct_arrest.PERP_RACE == race)
                if boro_precinct_arrest[select].shape[0]>0:
                    districts_table["Arr_A"+age_group+"//R"+race+"//S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                    districts_table["Arr_A"+age_group+"//R"+race+"//S"+sex] = districts_table["Arr_A"+age_group+"//R"+race+"//S"+sex].fillna(0)
                else:
                    districts_table["Arr_A"+age_group+"//R"+race+"//S"+sex] = 0

In [131]:
 for sex in boro_precinct_arrest.PERP_SEX.unique():
    for crime in ['F', 'M', 'V', 'I']:
        for race in boro_precinct_arrest.PERP_RACE.unique():
            select = (boro_precinct_arrest.PERP_SEX == sex)&(boro_precinct_arrest.LAW_CAT_CD == crime)&(boro_precinct_arrest.PERP_RACE == race)
            if boro_precinct_arrest[select].shape[0]>0:
                districts_table["C"+crime+"//R"+race+"//S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                districts_table["C"+crime+"//R"+race+"//S"+sex] = districts_table["C"+crime+"//R"+race+"//S"+sex].fillna(0)
            else:
                districts_table["C"+crime+"//R"+race+"//S"+sex] = 0

In [132]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
    for crime in ['F', 'M', 'V', 'I']:
        for race in boro_precinct_arrest.PERP_RACE.unique():
            select = (boro_precinct_arrest.AGE_GROUP == age_group)&(boro_precinct_arrest.LAW_CAT_CD == crime)&(boro_precinct_arrest.PERP_RACE == race)
            if boro_precinct_arrest[select].shape[0]>0:
                districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race] = districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race].fillna(0)
            else:
                districts_table["Arr_A"+age_group+"//C"+crime+"//R"+race] = 0

In [133]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
    for sex in boro_precinct_arrest.PERP_SEX.unique():
        for crime in ['F', 'M', 'V', 'I']:
            select = (boro_precinct_arrest.PERP_SEX == sex)&(boro_precinct_arrest.AGE_GROUP == age_group)&(boro_precinct_arrest.LAW_CAT_CD == crime)
            if boro_precinct_arrest[select].shape[0]>0:
                districts_table["Arr_A"+age_group+"//C"+crime+"//S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                districts_table["Arr_A"+age_group+"//C"+crime+"//S"+sex] = districts_table["Arr_A"+age_group+"//C"+crime+"//S"+sex].fillna(0)
            else:
                districts_table["Arr_A"+age_group+"//C"+crime+"//S"+sex] = 0

In [134]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
    for sex in boro_precinct_arrest.PERP_SEX.unique():
            select = (boro_precinct_arrest.PERP_SEX == sex)&(boro_precinct_arrest.AGE_GROUP == age_group)
            districts_table["Arr_A"+age_group+"//S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
            districts_table["Arr_A"+age_group+"//S"+sex] = districts_table["Arr_A"+age_group+"//S"+sex].fillna(0)

In [135]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
        for crime in ['F', 'M', 'V', 'I']:
            select = (boro_precinct_arrest.AGE_GROUP == age_group)&(boro_precinct_arrest.LAW_CAT_CD == crime)
            districts_table["Arr_A"+age_group+"//C"+crime] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
            districts_table["Arr_A"+age_group+"//C"+crime] = districts_table["Arr_A"+age_group+"//C"+crime].fillna(0)

In [136]:
for sex in boro_precinct_arrest.PERP_SEX.unique():
        for crime in ['F', 'M', 'V', 'I']:
            select = (boro_precinct_arrest.PERP_SEX == sex)&(boro_precinct_arrest.LAW_CAT_CD == crime)
            districts_table["Arr_C"+crime+"//S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
            districts_table["Arr_C"+crime+"//S"+sex] = districts_table["Arr_C"+crime+"//S"+sex].fillna(0)

In [137]:
for age_group in boro_precinct_arrest.AGE_GROUP.unique():  
    for race in boro_precinct_arrest.PERP_RACE.unique():
            select = (boro_precinct_arrest.AGE_GROUP == age_group)&(boro_precinct_arrest.PERP_RACE == race)
            if boro_precinct_arrest[select].shape[0]>0:
                    districts_table["Arr_A"+age_group+"//R"+race] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                    districts_table["Arr_A"+age_group+"//R"+race] = districts_table["Arr_A"+age_group+"//R"+race].fillna(0)
            else:
                    districts_table["Arr_A"+age_group+"//R"+race] = 0

In [138]:
for sex in boro_precinct_arrest.PERP_SEX.unique():
            for race in boro_precinct_arrest.PERP_RACE.unique():
                select = (boro_precinct_arrest.PERP_SEX == sex)&(boro_precinct_arrest.PERP_RACE == race)
                if boro_precinct_arrest[select].shape[0]>0:
                    districts_table["Arr_R"+race+"//S"+sex] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                    districts_table["Arr_R"+race+"//S"+sex] = districts_table["Arr_R"+race+"//S"+sex].fillna(0)
                else:
                    districts_table["Arr_R"+race+"//S"+sex] = 0

In [139]:
for crime in ['F', 'M', 'V', 'I']:
        for race in boro_precinct_arrest.PERP_RACE.unique():
            select = (boro_precinct_arrest.LAW_CAT_CD == crime)&(boro_precinct_arrest.PERP_RACE == race)
            if boro_precinct_arrest[select].shape[0]>0:
                districts_table["Arr_C"+crime+"//R"+race] = boro_precinct_arrest[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
                districts_table["Arr_C"+crime+"//R"+race] = districts_table["Arr_C"+crime+"//R"+race].fillna(0)
            else:
                districts_table["Arr_C"+crime+"//R"+race] = 0

### PI information

In [140]:
pnts = gp.GeoDataFrame(geometry=boro_PI["the_geom"], index=boro_PI.index)
pnts = pnts.assign(**{str(key): pnts.within(geom) for key, geom in districts.geometry.items()})

Drop PI outside city

In [141]:
pnts.sum(axis=1)[pnts.sum(axis=1)==0].index

Int64Index([    6,    21,    24,    26,    30,    34,    35,    37,    41,
               46,
            ...
            20010, 20012, 20013, 20015, 20016, 20018, 20019, 20020, 20021,
            20027],
           dtype='int64', length=1800)

In [142]:
boro_PI.drop(index = pnts.sum(axis=1)[pnts.sum(axis=1)==0].index, inplace = True)
pnts.drop(index = pnts.sum(axis=1)[pnts.sum(axis=1)==0].index, inplace = True)

In [143]:
boro_PI["boro_cd"] = pnts.apply(lambda x: eval(x[x==True].index[0])[1], axis =1)

In [144]:
districts_table["PI/Tot"] = boro_PI.groupby(["BOROUGH","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)

In [145]:
boro_PI.FACI_DOM = boro_PI.FACI_DOM.fillna("unknown")

In [146]:
for dom in boro_PI.FACI_DOM.unique():
    select = boro_PI.FACI_DOM == dom
    districts_table["PI/Tot_D"+str(dom)] = boro_PI[select].groupby(["BOROUGH","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
    districts_table["PI/Tot_D"+str(dom)] = districts_table["PI/Tot_D"+str(dom)].fillna(0)

### Schools information

In [147]:
pnts = gp.GeoDataFrame(geometry=schools["geometry"], index=schools.index)
pnts = pnts.assign(**{str(key): pnts.within(geom) for key, geom in districts.geometry.items()})

Drop schools outside city

In [148]:
pnts.sum(axis=1)[pnts.sum(axis=1)==0].index

Int64Index([1555, 1556], dtype='int64')

In [149]:
schools.drop(index = pnts.sum(axis=1)[pnts.sum(axis=1)==0].index, inplace = True)
pnts.drop(index = pnts.sum(axis=1)[pnts.sum(axis=1)==0].index, inplace = True)

In [150]:
schools["boro_cd"] = pnts.apply(lambda x: eval(x[x==True].index[0])[1], axis =1)
schools["boro_name"] = pnts.apply(lambda x: eval(x[x==True].index[0])[0], axis =1)

In [151]:
districts_table["Schools/Tot"] = schools.groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)

In [152]:
for type in schools.SCH_TYPE.unique():
    select = schools.SCH_TYPE == type
    districts_table["Schools/Tot_"+type] = schools[select].groupby(["boro_name","boro_cd"]).apply(lambda x: x.shape[0]/districts_table.Population_2019.loc[x.name]*100)
    districts_table["Schools/Tot_"+type] = districts_table["Schools/Tot_"+type].fillna(0)

### Demographic information [6]

In [153]:
demo = pd.read_csv(path + "Demo_2016.csv",sep =";", index_col=0)

In [154]:
districts_table = districts_table.reset_index()

$\text{ratio_demo_2016} = \frac{\text{demo_cat}}{\text{tot2016}}$

$\text{demo_2019} = \text{ratio_demo_2016}*\text{tot2019}$

$\text{demo_boro_cd_2019} = \text{demo_2019}*\frac{\text{tot_boro_cd_2019}}{\text{tot2019}}= \text{ratio_demo_2016}*\text{tot_boro_cd_2019}$

In [155]:
demo.rename({"StatenIsland":"Staten Island"}, inplace = True)

In [156]:
for boro in districts_table.Borough:
    for cd in districts_table.boro_cd:
        for col in demo.columns:
            districts_table.loc[(districts_table.Borough == boro) & (districts_table.boro_cd == cd),"demo_"+col] = float(demo.loc[boro,col].replace(",",""))*districts_table.loc[(districts_table.Borough == boro) & (districts_table.boro_cd == cd),"Population_2019"]

In [157]:
districts_table.head()

,Borough,boro_cd,Population_2019,CD Name,Arrest,Arr_SM,Arr_SF,Arr_A25-44,Arr_A18-24,Arr_A45-64,...,demo_Whitealone,demo_BlackorAfricanAmericanalone,demo_AmericanIndianandAlaskaNativealone,demo_Asianalone,demo_NativeHawaiianandOtherPacificIslanderalone,demo_Someotherracealone,demo_Twoormoreraces.2,demo_TworacesincludingSomeotherrace,demo_TworacesexcludingSomeotherraceandThreeormoreraces,demo_Totalhousingunits
0,Bronx,1,97192.183333,"Melrose, Mott Haven, Port Morris",5.543656,4.543575,1.000080,3.019790,1.011398,1.213061,...,9.328396e+12,2.866036e+13,2.085514e+11,3.475022e+12,3.436395e+10,6.529830e+11,9.735554e+11,1.851461e+11,7.884095e+11,3.533813e+13
1,Bronx,2,58333.850000,"Hunts Point, Longwood",4.138249,3.455969,0.682280,2.137695,1.160561,0.599995,...,5.598817e+12,1.720168e+13,1.251706e+11,2.085676e+12,2.062492e+10,3.919143e+11,5.843190e+11,1.111230e+11,4.731961e+11,2.120962e+13
2,Bronx,3,89969.333333,"Morrisania, Crotona Park East",3.198868,2.557538,0.641330,1.650562,0.711353,0.654668,...,8.635155e+12,2.653046e+13,1.930529e+11,3.216775e+12,3.181019e+10,6.044564e+11,9.012055e+11,1.713869e+11,7.298187e+11,3.271197e+13
3,Bronx,4,160477.050000,"Highbridge, Concourse Village",3.299537,2.724377,0.575160,1.785925,0.707266,0.642459,...,1.540241e+13,4.732201e+13,3.443457e+11,5.737718e+12,5.673939e+10,1.078161e+12,1.607468e+12,3.057004e+11,1.301768e+12,5.834789e+13
4,Bronx,5,134259.416667,"University Hts., Fordham, Mt. Hope",2.716383,2.168191,0.548192,1.405488,0.592137,0.590648,...,1.288607e+13,3.959087e+13,2.880889e+11,4.800329e+12,4.746970e+10,9.020181e+11,1.344851e+12,2.557572e+11,1.089094e+12,4.881541e+13


## Weather table [7]

In [158]:
l = ["january","february","march","april","may","june","july","august","september"]
temp = pd.DataFrame()
for i in range(0,9):
    table = pd.read_csv(path+l[i]+"_2019.txt", delim_whitespace =  True)[["DY","AVG","DEP"]]
    table.rename(columns = {"DY":"DAY","AVG":"Temperature"}, inplace = True)
    table["MONTH"]=i+1
    temp = pd.concat([table,temp], ignore_index = True)

In [159]:
boro_precinct_arrest = pd.merge(boro_precinct_arrest,temp, on = ["MONTH","DAY"])

## Save tables

In [160]:
schools.to_csv(path + "Modified\\Schools.csv")
boro_precinct_arrest.to_csv(path + "Modified\\boro_precinct_arrest.csv")
boro_PI.to_csv(path + "Modified\\boro_PI.csv")
districts.reset_index().to_file(path + "Modified\\Districts.shp")
districts_table.to_csv(path + "Modified\\Districts_table.csv",index = False)

## References

[1] Police Department (NYPD), “NYPD Arrest Data (Year to Date),” Cityofnewyork.us, 05-Jun-2018. [Online]. Available: http://data.cityofnewyork.us/Public-Safety/NYPD-Arrest-Data-Year-to-Date-/uip8-fykc. [Accessed: 05-Dec-2019].

[2]
Open Calgary, “Points Of Interest | NYC Open Data,” NYC Open Data, 2018. [Online]. Available: http://data.cityofnewyork.us/City-Government/Points-Of-Interest/rxuy-2muj. [Accessed: 05-Dec-2019].

[3]
Department of Education (DOE, “School Point Locations,” Cityofnewyork.us, 22-Sep-2011. [Online]. Available: http://data.cityofnewyork.us/Education/School-Point-Locations/jfju-ynrr. [Accessed: 05-Dec-2019].

[4]
Open Calgary, “Community Districts | NYC Open Data,” NYC Open Data, 2018. [Online]. Available: http://data.cityofnewyork.us/City-Government/Community-Districts/yfnk-k7r4. [Accessed: 05-Dec-2019].

[5]
Department of City Planning (DCP, “New York City Population By Community Districts,” Cityofnewyork.us, 20-Feb-2013. [Online]. Available: http://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2. [Accessed: 05-Dec-2019].

[6]
Department of City Planning (DCP, “Demographic and Housing Profiles by Borough,” Cityofnewyork.us, 09-Aug-2011. [Online]. Available: http://data.cityofnewyork.us/City-Government/Demographic-and-Housing-Profiles-by-Borough/cu9u-3r5e. [Accessed: 05-Dec-2019].

[7]
National Weather Service Corporate Image Web Team, “National Weather Service Climate,” Weather.gov, 2019. [Online]. Available: https://w2.weather.gov/climate/index.php?wfo=okx.
